In [1]:
import pandas as pd
import utility

pricesDataFolder = "prices_data/"
dfH4 = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"H4/[SP500]_H4_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)

dfH4 = dfH4.rename(
    columns = {
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)

dfH4['datetime'] = pd.to_datetime(dfH4['<DATE>'] + ' ' + dfH4['<TIME>'])
dfH4 = dfH4.drop(['<TICKVOL>', '<VOL>', '<SPREAD>', '<DATE>', '<TIME>'], axis=1)
dfH4 = utility.heikinashi(dfH4) #add heikin ashi candles
dfH4["shortTermMA"] = dfH4["close"].rolling(window=50).mean() # add moving average 50
dfH4["longTermMA"] = dfH4["close"].rolling(window=200).mean() # add moving average 200
dfH4 = utility.addHACandleColor(dfH4)

In [2]:

from actor_critic_training import actorCritictrainingLoop
from actor_critic.trading_agent.agent import Agent
from strategies.Heikin_Ashi_Moving_Average_Strategy import HeikinAshiMovingAverage

agent = Agent()
strategy = HeikinAshiMovingAverage(agent, useSR="SRbyRL", useUpdateSl=True, uselongTermMA=False)

actorCritictrainingLoop(dfH4, strategy)

2024-04-01 12:58:40.557503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 12:58:41.936888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-01 12:58:44.174502: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 12:58:44.384197: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required li

episode: 0, score: 27.221206665039062, avg_score: 27.221206665039062
episode: 1, score: 29.5, avg_score: 28.36060333251953
episode: 2, score: 29.5, avg_score: 28.740402221679688
episode: 3, score: 29.5, avg_score: 28.930301666259766


KeyboardInterrupt: 